In [3]:
import matplotlib.pyplot as plt
from numpy import arange, exp, pi, zeros, correlate, array
from scipy.signal import find_peaks
from pathlib import Path
from os import listdir
import ipywidgets as widgets
from IPython.display import display
%matplotlib widget

In [4]:
def auto_correlation(type, data):
    """Doing correlation to the signal itself using designed mathematics equation. Time shifted to the length of data.

    Parameters
    ----------
    type : String
        Function type: normal for using the direct correlation, inversefourier using magnitude spectrum
    data : 1D array
        Input of the function, data signal

    Returns
    -------
    1D array
        Return of the autocorrelation
    """
    result = array(zeros(len(data)), dtype=complex)
    magnitude_spectrum = array(zeros(len(data)), dtype=complex)
    if type == "normal":
        return [x/len(data) for x in correlate(data, data, mode="same")]
    elif type == "inversefourier":
        for i in range(len(data)):
            for j in range(len(data)):
                magnitude_spectrum[i] += (data[j]**2)*exp(-2j*pi*i*j/len(data))
        
        for i in range(len(data)):
            for j in range(len(data)):
                result[i] += (magnitude_spectrum[j]**2)*exp(2j*pi*i*j/len(magnitude_spectrum))
            result[i] = result[i]/len(magnitude_spectrum)
        return result
    else: raise Exception("Please check the available type!") 

In [6]:
def cross_correlation(type, first_data, second_data):
    """Doing correlation on two signal using designed mathematical equation. Time shifted to the first signal. 

    Parameters
    ----------
    type : String
        Function type: normal for using the direct correlation, inversefourier using magnitude spectrum
    firstdata : 1D Array
        Base signal for correlation. The signal that doesn't move on the line.
    seconddata : 1D Array
        Signal for the correlation, signal move across the first signal.

    Returns
    -------
    1D Array
        Return of the function with length of the first signal.
    """

    if len(first_data) != len(second_data):
        raise Exception("Please select the same length of the first and second data!")

    data_length = len(first_data)

    if type == "normal":
        return [x/data_length for x in correlate(first_data, second_data, mode="same")]

In [8]:
def ecg_peak_finding(type, data):
    """Finding the peak position of the electrocardiogram signal based on the correlation mathematical function method.

    Parameters
    ----------
    type : String
        Function type: normal for using the direct correlation, inversefourier using magnitude spectrum
    data : 1D Array
        Input array of electrocardiogram signal

    Returns
    -------
    Tuple [1D Array, 1D Array]
        Return the peak position and the peak value of the electrocardiogram signal.
    """
    correlation = auto_correlation(type, data)
    peak_position = find_peaks(correlation, distance=100, height=0.1)[0]

    peak_value = [correlation[i] for i in peak_position]
    return peak_position, peak_value

In [11]:
def on_change_select_first_signal(change):
    with out:
        texts0 = [[], []] # X(sequence) and Y(ecg signal) value
        with open(select_first_signal.get_interact_value(), "r") as file:
            text_file = file.readlines()
        texts0[0] = arange(len(text_file))
        try:
            for text in text_file: 
                texts0[1].append(float(text.replace("\n", "")))
        except ValueError:
            print("Please select single column data file.")
            return
        fig0_ax0.clear()
        fig0_ax0.plot(texts0[0], texts0[1])
        first_signal_length.value = f"{len(texts0[0])}"

In [12]:
def on_change_select_second_signal(change):
    with out:
        texts0 = [[], []] # X(sequence) and Y(ecg signal) value
        with open(select_second_signal.get_interact_value(), "r") as file:
            text_file = file.readlines()
        texts0[0] = arange(len(text_file))
        try:
            for text in text_file: 
                texts0[1].append(float(text.replace("\n", "")))
        except ValueError:
            print("Please select single column data file.")
            return
        fig0_ax1.clear()
        fig0_ax1.plot(texts0[0], texts0[1], color="green")
        second_signal_length.value = f"{len(texts0[0])}"

In [15]:
print("Please select the same data length for first and second signal")
files = listdir(Path().absolute())

select_first_signal = widgets.Select(
    options=files,
    description='Select First File:',
    disabled=False
)

first_signal_length = widgets.Text(value='0', disabled=True)

select_second_signal = widgets.Select(
    options=files,
    description='Select Second File:',
    disabled=False
)

second_signal_length = widgets.Text(value="0", disabled=True)
out = widgets.Output()

select_first_signal.observe(on_change_select_first_signal)
select_second_signal.observe(on_change_select_second_signal)

display(select_first_signal, first_signal_length, select_second_signal, second_signal_length, out)

fig0 = plt.figure()
fig0_ax0 = fig0.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude")
fig0_ax1 = fig0_ax0.twinx()
fig0_ax1.set_ylabel(ylabel="amplitude")
plt.show()

Please select the same data length for first and second signal


Select(description='Select First File:', options=('ACF CCF Fix.py', 'Reflected Sinus 10 Hz 512 Data.txt', 'ACF…

Text(value='0', disabled=True)

Select(description='Select Second File:', options=('ACF CCF Fix.py', 'Reflected Sinus 10 Hz 512 Data.txt', 'AC…

Text(value='0', disabled=True)

Output()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
texts0 = [[], []] # X(sequence) and Y(ecg signal) value
texts1 = [[], []] # X(sequence) and Y(ecg signal) value

with open(select_first_signal.get_interact_value(), "r") as file:
    text_file = file.readlines()
texts0[0] = arange(len(text_file))
for text in text_file: 
    texts0[1].append(float(text.replace("\n", "")))

with open(select_second_signal.get_interact_value(), "r") as file:
    text_file = file.readlines()
texts1[0] = arange(len(text_file))
for text in text_file: 
    texts1[1].append(float(text.replace("\n", "")))


In [ ]:
fig = plt.figure()
ax0 = fig.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude", title="Auto Correlation")
ax0.plot(texts0[0], [abs(x) for x in auto_correlation("normal", texts0[1])], label="Phase Shift")
ax0.legend()
ax1 = ax0.twinx()
ax1.set_ylabel(ylabel="amplitude")
ax1.plot(texts0[0], [abs(x) for x in auto_correlation("inversefourier", texts0[1])], label="Inverse Fourier", color="green")
ax1.legend()
plt.show()

In [29]:
fig = plt.figure()
ax0 = fig.add_axes([0.1,0.1,0.85,0.85], xlabel="sequence", ylabel="amplitude")
ax0.plot([x - (len(texts0[0]))//2 for x in texts0[0]], cross_correlation("normal", texts0[1], texts1[1]), label="Correlated Signal")
ax0.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …